# CCSD theory for a closed-shell reference

In this notebook we will use Wick\&d to generate and implement equations for the spin-integrated CCSD method.
To simplify this notebook some of the utility functions are imported from the file `examples_helpers_spin_integrated.py`.
In this example, we run a CCSD computation on the H<sub>2</sub>O molecule, reading all the relevant information from the file `sr-h2o-cc-pvdz.npy`.

In [1]:
import time
import wicked as w
import numpy as np
from examples_helpers_spin_integrated import *
import itertools

## Read calculation information (integrals, number of orbitals)

We start by reading information about the reference state, integrals, and denominators from the file `sr-h2o-cc-pvdz.npy`. The variable `H` is a dictionary that holds the blocks of the Hamiltonian **normal-ordered** with respect to the Hartree–Fock determinant. `invD` similarly is a dictionary that stores the denominators $(\epsilon_i + \epsilon_j + \ldots - \epsilon_a - \epsilon_b - \ldots)^{-1}$.

In [2]:
molecule = "sr-h2o-cc-pvdz"
with open(f"{molecule}.npy", "rb") as f:
    Eref = np.load(f)
    nocc, nvir = np.load(f)
    H = np.load(f, allow_pickle=True).item()
invD = compute_inverse_denominators(H, nocc, nvir, 2)

## Define orbital spaces and the Hamiltonian and cluster operators

Here we define the cluster operator (`Top`) and the Hamiltonian (`Hop`) that will be used to derive the CCSD equations. We also define the similarity-transformed Hamiltonian $\bar{H}$ truncated at the four-nested commutator:

\begin{equation}
\bar{H} = \hat{H} + [\hat{H},\hat{T}] + \frac{1}{2} [[\hat{H},\hat{T}],\hat{T}]
+ \frac{1}{6} [[[\hat{H},\hat{T}],\hat{T}],\hat{T}]
+ \frac{1}{24} [[[[\hat{H},\hat{T}],\hat{T}],\hat{T}],\hat{T}] + \ldots
\end{equation}

To do spin-integration, we simply introduce separate orbital spaces for alpha and beta spinorbitals. We use the convention that small cap letters denote alpha spinorbitals and big cap letters denote beta spinorbitals.

We include all the $M_S$-conserving operators in $\hat{T}$ and $\hat{H}$. We use the optional argument `unique=True` in the convenience function `w.op` to prevent combinatorically equivalent operators from being included more than once, which would result in incorrect factors.

In [13]:
w.reset_space()
# alpha
w.add_space('o', 'fermion', 'occupied', list('ijklmn'))
w.add_space('v', 'fermion', 'unoccupied', list('abcdef'))
# beta
w.add_space('O', 'fermion', 'occupied', list('IJKLMN'))
w.add_space('V', 'fermion', 'unoccupied', list('ABCDEF'))

Top = w.op("T", ["v+ o", "V+ O", "v+ v+ o o", "V+ V+ O O", "V+ v+ O o"], unique=True)
Hops = []
for i in itertools.product(['v+', 'o+'],['v', 'o']):
    Hops.append(' '.join(i))
for i in itertools.product(['V+', 'O+'],['V', 'O']):
    Hops.append(' '.join(i))
for i in itertools.product(['v+', 'o+'],['v+', 'o+'], ['v', 'o'], ['v', 'o']):
    Hops.append(' '.join(i))
for i in itertools.product(['V+', 'O+'],['V+', 'O+'], ['V', 'O'], ['V', 'O']):
    Hops.append(' '.join(i))
for i in itertools.product(['v+', 'o+'],['V+', 'O+'], ['v', 'o'], ['V', 'O']):
    Hops.append(' '.join(i))
Hop = w.op("H", Hops, unique=True)
Hbar = w.bch_series(Hop, Top, 4)

In the following lines, we apply Wick's theorem to simplify the similarity-transformed Hamiltonian $\bar{H}$ computing all contributions ranging from operator rank 0 to 4 (double substitutions).
Then we convert all the terms into many-body equations accumulated into the residual `R`.

In [14]:
wt = w.WickTheorem()
expr = wt.contract(w.rational(1), Hbar, 0, 4)
mbeq = expr.to_manybody_equation("R")

Here we finally generate the CCSD equations. We use the utility function `generate_equation` to extract the equations corresponding to a given number of creation and annihilation operators and generated Python functions that we then define with the command `exec`

In [15]:
energy_eq = generate_equation(mbeq, 0, 0, '|')
t1_aa_eq = generate_equation(mbeq, 1, 1, 'o|v')
t1_bb_eq = generate_equation(mbeq, 1, 1, 'O|V')
t2_aaaa_eq = generate_equation(mbeq, 2, 2, 'oo|vv')
t2_bbbb_eq = generate_equation(mbeq, 2, 2, 'OO|VV')
t2_abab_eq = generate_equation(mbeq, 2, 2, 'oO|Vv')
exec(energy_eq)
exec(t1_aa_eq)
exec(t1_bb_eq)
exec(t2_aaaa_eq)
exec(t2_bbbb_eq)
exec(t2_abab_eq)

## CCSD algorithm

Here we code a simple loop in which we evaluate the energy and residuals of the CCSD equations and update the amplitudes

In [16]:
T = {"ov": np.zeros((nocc, nvir)), "OV": np.zeros((nocc,nvir)),\
      "oovv": np.zeros((nocc, nocc, nvir, nvir)), "OOVV": np.zeros((nocc, nocc, nvir, nvir)), "oOvV": np.zeros((nocc, nocc, nvir, nvir))}

header = "Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.perf_counter()

maxiter = 50
Ecorr_ref = -0.223910077025

for i in range(maxiter):
    # 1. compute energy and residuals
    R = {}
    Ecorr_w = evaluate_residual_(H, T)
    Etot_w = Eref + Ecorr_w
    R["ov"] = evaluate_residual_ov(H, T)
    R["OV"] = evaluate_residual_OV(H, T)
    Roovv = evaluate_residual_oovv(H, T)
    R["oovv"] = antisymmetrize_residual_2_2(Roovv, nocc, nvir)
    ROOVV = evaluate_residual_OOVV(H, T)
    R["OOVV"] = antisymmetrize_residual_2_2(ROOVV, nocc, nvir)
    R["oOvV"] = evaluate_residual_oOvV(H, T) # no need to antisymmetrize

    # 2. amplitude update
    update_cc_amplitudes(T, R, invD, 2)

    # 3. check for convergence
    norm_R = np.sqrt(np.linalg.norm(R["ov"]) ** 2 + np.linalg.norm(R["OV"]) ** 2 + np.linalg.norm(R["oovv"]) ** 2 + np.linalg.norm(R["OOVV"]) ** 2 + np.linalg.norm(R["oOvV"]) ** 2)
    print(f"{i:3d}    {Etot_w:+.12f}    {Ecorr_w:+.12f}    {norm_R:e}")
    if norm_R < 1.0e-8:
        break

end = time.perf_counter()
t = end - start

print("-" * len(header))
print(f"CCSD total energy                   {Etot_w:+.12f} [Eh]")
print(f"CCSD correlation energy             {Ecorr_w:+.12f} [Eh]")
print(f"Reference CCSD correlation energy   {Ecorr_ref:+.12f} [Eh]")
print(f"Error                               {Ecorr_w - Ecorr_ref:+.12e} [Eh]")
print(f"Timing                              {t:+.12e} [s]")
assert np.isclose(Ecorr_w, Ecorr_ref)


-----------------------------------------------------------
Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       
-----------------------------------------------------------
  0    -75.989795787487    +0.000000000000    1.299100e+00
  1    -76.204143461765    -0.214347674279    1.722778e-01
  2    -76.207264973707    -0.217469186220    4.253534e-02
  3    -76.211865730825    -0.222069943338    1.868229e-02
  4    -76.212864167040    -0.223068379554    8.304716e-03
  5    -76.213350138227    -0.223554350740    3.961553e-03
  6    -76.213541063949    -0.223745276462    1.959698e-03
  7    -76.213628949546    -0.223833162059    1.002226e-03
  8    -76.213668687064    -0.223872899577    5.318052e-04
  9    -76.213687566001    -0.223891778515    2.909445e-04
 10    -76.213696641243    -0.223900853756    1.637721e-04
 11    -76.213701122859    -0.223905335373    9.419992e-05
 12    -76.213703376304    -0.223907588817    5.510109e-05
 13    -76.213704534002    -0.223908746515    3.26144